## Malaria Classification on Blood Cells images

### This is a work in progress

Splitted the dataset to 3 sets for training, testing and validation. validation is 0.1 of the entire data. The rest was splitted 0.8 and 0.2 respectevily
Made several tuning.
The simplicity of the model and task is great for everyone to learn pyTorch and get intuitation about DNN and this dataset. So everyone are welcomed and get interested about this toy and wonderful dataset.


any error in the code is mine

In [5]:
#########################################                                
#
#         Anopheles Project
#        Detecting malaria in blood cells images
#       Done by Efi Eisenberg as a self-taught project
#
#    Everybody - It will be super cool if you find some way to make the network 
#   work better. Enjoy
#
####################################################################################


import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

TRAIN_PATH = "/home/evilberg/Documents/Malaria Project/Data/Train/"
TEST_PATH = "/home/evilberg/Documents/Malaria Project/Data/Test/"
VALIDATION_PATH = "/home/evilberg/Documents/Malaria Project/Data/Validation/"

In [6]:
## Preprocessing
## Normalizing the data found better on original variance than unit variance

transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.1395, 0.2531, 0.3396], [1, 1, 1])])  

train_data = datasets.ImageFolder(root=TRAIN_PATH,
                                  transform=transform)

test_data = datasets.ImageFolder(root=TEST_PATH,
                                  transform=transform)

val_data = datasets.ImageFolder(root=VALIDATION_PATH, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=2,
                                             shuffle=True,
                                             num_workers=2)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=2,
                                          shuffle=False,
                                          num_workers=2)

val_loader = torch.utils.data.DataLoader(val_data,
                                          batch_size=2,
                                          shuffle=False,
                                          num_workers=2)

classes = ('Parasitized', 'Uninfected')

In [7]:
## Defining the network

class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation : y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.elu(self.conv1(x)))
        x = self.pool(F.elu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = self.fc3(x)

        return x

In [8]:
## Training the network

net = ConvNet()

print(net)
param = list(net.parameters())
print("len of param: ", len(param))


criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)

for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the input
        inputs, labels = data

        # zero the parameters
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # print every 200 minibatches
            print("[{}, {}], Loss: {}".format(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)
('len of param: ', 10)
[1, 200], Loss: 1.39150323212
[1, 400], Loss: 1.36806358844
[1, 600], Loss: 1.37107931495
[1, 800], Loss: 1.33920432895
[1, 1000], Loss: 1.34845977813
[1, 1200], Loss: 1.31294670373
[1, 1400], Loss: 1.34555169165
[1, 1600], Loss: 1.34662138909
[1, 1800], Loss: 1.24064220279
[1, 2000], Loss: 1.2772927317
[1, 2200], Loss: 1.2591595912
[1, 2400], Loss: 1.29976348355
[1, 2600], Loss: 1.27086585015
[1, 2800], Loss: 1.29115883827
[1, 3000], Loss: 1.19821534872
[1, 3200], Loss: 1.19208962157
[1, 3400], Loss: 1.1654206492
[1, 3600], Loss: 1.09754492819
[1, 3800], Loss: 1.0771438245

In [33]:
## Testing

correct = 0
total = 0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(correct)
print(total)
print(float(correct / total)
            
print('Accuracy of the network on %d test images: %d %%' % (len(test_loader),
    100 * correct / total))

for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

4700
4962
0.947198710198
Accuracy of the network on 2481 test images: 94 %
Accuracy of Parasitized : 95 %
Accuracy of Uninfected : 93 %


In [42]:
## validation

correct = 0
total = 0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

with torch.no_grad():
    for data in val_loader:
        image, labels = data
        outputs = net(image)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print(correct)
print(total)
print(float(correct) / total)
            

2596
2755
0.942286751361
